In [480]:
# Imports
import pandas as pd
import datetime
import sys

In [481]:
# Read Excel files

#  To change
folder_path = "DataSets/example-set/"

df_clients = pd.read_csv(folder_path + 'input_clients.csv')
df_instruments = pd.read_csv(folder_path + 'input_instruments.csv')
df_orders = pd.read_csv(folder_path + 'input_orders.csv')

# for i in df_orders.index:
#     # df_orders["Time"][i] = datetime.datetime.strptime(df_orders["Time"][i] , "%H:%M:%S").time()
#     df_orders.loc[i, "Time"] = datetime.datetime.strptime(df_orders["Time"][i] , "%H:%M:%S").time()

print("Clients")
print(df_clients.to_string())

print("Instruments")
print(df_instruments.to_string())

print("Orders")
print(df_orders.to_string())

Clients
  ClientID   Currencies PositionCheck  Rating
0        A      USD,SGD             Y       1
1        B  USD,SGD,JPY             N       2
2        C          SGD             Y       3
3        D          USD             Y       4
4        E          SGD             N       5
Instruments
  InstrumentID Currency  LotSize
0          SIA      SGD      100
Orders
        Time OrderID Instrument  Quantity Client   Price  Side
0    9:00:01      A1        SIA      1500      A  Market   Buy
1    9:02:00      B1        SIA      4500      B    32.1  Sell
2    9:05:00      C1        SIA       100      C      32   Buy
3    9:10:00      D1        SIA       300      D  Market  Sell
4    9:29:01      B2        SIA         5      B    32.1  Sell
5    9:29:02      E1        SIA      1000      E      32  Sell
6    9:29:03      A2        SIA       800      A    31.9   Buy
7    9:30:01      C2        SIA       100      C  Market  Sell
8    9:40:00      B3        SIA       500      B    32.2  Sell
9

In [482]:
class Client:
    def __init__(self, client_id, currencies, position_check, rating):
        self.client_id = client_id
        self.currencies =currencies
        self.position_check = position_check == "Y"
        self.rating = rating
        
        self.positions = {}

    def __str__(self):
        return "{Client}: " + self.__dict__.__str__()
    

    def __repr__(self):
        return self.__str__()

clients = []

# Add clients
for i in df_clients.index:
    clients.append(Client(
        df_clients["ClientID"][i],
        df_clients["Currencies"][i],
        df_clients["PositionCheck"][i],
        df_clients["Rating"][i]
    ))

print(clients)

[{Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {}}, {Client}: {'client_id': 'B', 'currencies': 'USD,SGD,JPY', 'position_check': False, 'rating': 2, 'positions': {}}, {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, {Client}: {'client_id': 'D', 'currencies': 'USD', 'position_check': True, 'rating': 4, 'positions': {}}, {Client}: {'client_id': 'E', 'currencies': 'SGD', 'position_check': False, 'rating': 5, 'positions': {}}]


In [483]:
class Instrument:
    def __init__(self, instrumentID, currency, lotsize):
        self.instrumentID = instrumentID
        self.currency = currency
        self.lotsize = lotsize

    def __str__(self):
        return "{Instrument}: " + self.__dict__.__str__()
    

    def __repr__(self) -> str:
        return self.__str__()

instruments = []

# Add instruments
for i in df_instruments.index:
    instruments.append(Instrument(
        df_instruments["InstrumentID"][i],
        df_instruments["Currency"][i],
        df_instruments["LotSize"][i]
    ))

print(instruments)

[{Instrument}: {'instrumentID': 'SIA', 'currency': 'SGD', 'lotsize': 100}]


In [484]:
class Order:
    def __init__(self, time, order_id, client, instrument, side, price, quantity):
        self.time = time
        self.order_id = order_id
        self.client = client
        self.instrument = instrument
        self.side = side
        self.price = price if price == "Market" else float(price)
        self.quantity = float(quantity)
        self.is_market = price.lower() == "market"

    def __str__(self):
        return "{Order}: " + self.__dict__.__str__()
    

    def __repr__(self):
        return self.__str__()

orders = []

for i in df_orders.index:
    client = [c for c in clients if c.client_id == df_orders["Client"][i]][0]

    orders.append(
        Order(
            datetime.datetime.strptime(df_orders["Time"][i], "%H:%M:%S").time(),
            df_orders["OrderID"][i],
            client,
            df_orders["Instrument"][i],
            df_orders["Side"][i],
            df_orders["Price"][i],
            df_orders["Quantity"][i],
        )
    )

print(orders)
        

[{Order}: {'time': datetime.time(9, 0, 1), 'order_id': 'A1', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 'Market', 'quantity': 1500.0, 'is_market': True}, {Order}: {'time': datetime.time(9, 2), 'order_id': 'B1', 'client': {Client}: {'client_id': 'B', 'currencies': 'USD,SGD,JPY', 'position_check': False, 'rating': 2, 'positions': {}}, 'instrument': 'SIA', 'side': 'Sell', 'price': 32.1, 'quantity': 4500.0, 'is_market': False}, {Order}: {'time': datetime.time(9, 5), 'order_id': 'C1', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.0, 'quantity': 100.0, 'is_market': False}, {Order}: {'time': datetime.time(9, 10), 'order_id': 'D1', 'client': {Client}: {'client_id': 'D', 'currencies': 'USD', 'position_check': True, 'rating': 4, 'positions': {}}, 'instrument': '

In [485]:
class OrderBook:
    def __init__(self):
        self.orders = {
            "buy": [],
            "sell": []
        }

    def sort_orders(self):
        for side in self.orders.keys():
            order = self.orders[side]
            order.sort(key=lambda o : o.time)
            order.sort(key=lambda o : o.client.rating)
            order.sort(key=lambda o : 999999 if o.price == "Market" else o.price, reverse=side.lower() == "buy")

        return self

    def add_order(self, order):
            if order.side == "Buy":
                self.orders["buy"].append(order)
            else:
                self.orders["sell"].append(order)

    def get_best_buy(self):
        return self.orders["buy"][0]

    def get_next_best_buy(self):
        i = 0
        while self.orders["buy"][i].is_market and i < len(self.orders["buy"]):
            i += 1
        
        if self.orders["buy"][i].is_market:
            return None

        return self.orders["buy"][i]

    def get_best_sell(self):
        return self.orders["sell"][0]

    def get_next_best_sell(self):
        i = 0
        while self.orders["sell"][i].is_market and i < len(self.orders["sell"]):
            i += 1
        
        if self.orders["sell"][i].is_market:
            return None

        return self.orders["sell"][i]

    def execute(self, buy, sell):
        qty_traded = 0
        if buy.quantity == sell.quantity:
            self.orders["buy"].remove(buy)
            self.orders["sell"].remove(sell)
            qty_traded = buy.quantity
        
        elif buy.quantity > sell.quantity:
            buy.quantity -= sell.quantity
            self.orders["sell"].remove(sell)
            qty_traded = sell.quantity
        
        else:
            self.orders["buy"].remove(buy)
            sell.quantity -= buy.quantity
            qty_traded = buy.quantity

        # Update clients
        buy.client.positions[buy.instrument] = buy.client.positions.get(buy.instrument, 0) + qty_traded
        sell.client.positions[sell.instrument] = sell.client.positions.get(sell.instrument, 0) - qty_traded

    def __str__(self):
        output = "---{OrderBook}---"
        for side in self.orders.keys():
            output += "\n-" + side
            for order in self.orders[side]:
                output += "\n" + order.__str__()

        return output + "---"
    

    def __repr__(self):
        return self.__str__()


In [486]:
# Validate orders
def matching_policies(orders, instruments, clients):
    new_orders = []

    for idx, order in enumerate(orders):
        idx_of_instrument = [i.instrumentID for i in instruments].index(order.instrument)
        # "REJECTED - INSTRUMENT NOT FOUND"
        if idx_of_instrument < 0:
            print(str(idx) + "REJECTED - INSTRUMENT NOT FOUND")
            continue

        instrumentID = instruments[idx_of_instrument]
        # "REJECTED - MISMATCH CURRENCY"
        if instruments[idx_of_instrument].currency not in order.client.currencies:
            print(str(idx) + "REJECTED - MISMATCH CURRENCY")
            continue

        # "REJECTED - INVALID LOT SIZE"
        if order.quantity % instruments[idx_of_instrument].lotsize != 0:
            print(str(idx) + "REJECTED - INVALID LOT SIZE")
            continue
        
        # "REJECTED - POSITION CHECK FAILED"
        # if order.client.position_check and order.client.positions.get(instrumentID, 0) - order.quantity < 0:
        #     print("REJECTED - POSITION CHECK FAILED")
        #     continue

        new_orders.append(order)

    return new_orders

                        
cleaned_orders = matching_policies(orders, instruments, clients)
print(cleaned_orders)

def validate_position(order):
        "REJECTED - POSITION CHECK FAILED"
        if order.client.position_check and order.side == "Sell" and order.client.positions.get(order.instrument, 0) - order.quantity < 0:
            print("REJECTED - POSITION CHECK FAILED")
            return False
        return True

3REJECTED - MISMATCH CURRENCY
4REJECTED - INVALID LOT SIZE
[{Order}: {'time': datetime.time(9, 0, 1), 'order_id': 'A1', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 'Market', 'quantity': 1500.0, 'is_market': True}, {Order}: {'time': datetime.time(9, 2), 'order_id': 'B1', 'client': {Client}: {'client_id': 'B', 'currencies': 'USD,SGD,JPY', 'position_check': False, 'rating': 2, 'positions': {}}, 'instrument': 'SIA', 'side': 'Sell', 'price': 32.1, 'quantity': 4500.0, 'is_market': False}, {Order}: {'time': datetime.time(9, 5), 'order_id': 'C1', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.0, 'quantity': 100.0, 'is_market': False}, {Order}: {'time': datetime.time(9, 29, 2), 'order_id': 'E1', 'client': {Client}: {'client_id': 'E', 'currencies': 'SGD', 'positio

In [487]:
# Tests

In [488]:
# Separate by time
order_book = {}

# Open auction (09:00 - 09:30)
open_orders = []
for order in cleaned_orders:
    if (order.time < datetime.datetime.strptime("9:30:00", "%H:%M:%S").time()):
        if not validate_position(order):
            # Failed position check
            # print(order)
            continue

        open_orders.append(order)

for order in open_orders:
    if order.instrument not in order_book:
        order_book[order.instrument] = OrderBook()

    order_book[order.instrument].add_order(order)

for instrument_id in order_book:
    order_book[instrument_id].sort_orders()
    
# print(order_book)

"""
Matching algorithm
"""
# 1. Buy and sell not market
    # 1.a Buy price >= sell price
        # Execute
    # 2.b Buy price < sell.price
        # Do not execute

# 2. Buy is market and sell is not market
    # Execute

# 3. Sell is market and buy is not market
    # Execute

# 4. Buy and sell market
    # Sell market and buy non market

trade_executed = True

while trade_executed:
    trade_executed = False
    for instrument_id in order_book:
        best_buy = order_book[instrument_id].get_best_buy()
        best_sell = order_book[instrument_id].get_best_sell()

        # print(best_buy)
        # print(best_sell)

        # 1. Buy and sell not market
        if not best_buy.is_market and not best_sell.is_market:
            # 1.a Buy price >= sell price
            if best_buy.price >= best_sell.price:
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True
                # Execute
            # 2.b Buy price < sell.price
                # Do not execute

        # 2. Buy is market and sell is not market
        elif best_buy.is_market:
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        # 3. Sell is market and buy is not market
        elif best_sell.is_market:
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        # 4. Buy and sell market
            # Sell non market and buy market

        else:
            if not (order_book[instrument_id].get_next_best_sell() is None):
                # Execute
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True

            elif not (order_book[instrument_id].get_next_best_buy() is None):
                # Execute
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True

            else:
                # All market orders
                pass

        print("###")
        print(order_book)
        print("###")
    # trade_executed = False
    
    
    

# 2. Buy i
print(clients)


###
{'SIA': ---{OrderBook}---
-buy
{Order}: {'time': datetime.time(9, 0, 1), 'order_id': 'A1', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {'SIA': 1000.0}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 'Market', 'quantity': 500.0, 'is_market': True}
{Order}: {'time': datetime.time(9, 5), 'order_id': 'C1', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.0, 'quantity': 100.0, 'is_market': False}
{Order}: {'time': datetime.time(9, 29, 3), 'order_id': 'A2', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {'SIA': 1000.0}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 31.9, 'quantity': 800.0, 'is_market': False}
-sell
{Order}: {'time': datetime.time(9, 2), 'order_id': 'B1', 'client': {Client}: {'client_id': 'B', 'currencies': 'USD,SGD,JPY', 'pos

In [489]:
# Continuous trading (09:30 - 16:00)
ct_orders = []
for order in orders:
    if (datetime.datetime.strptime("16:00:00", "%H:%M:%S").time() > order.time >= datetime.datetime.strptime("9:30:00", "%H:%M:%S").time()):
        ct_orders.append(order)


ct_orders.sort(key=lambda o : o.time)

"""
Matching algorithm
"""

for order in ct_orders:
    if not validate_position(order):
        # Failed position check
        continue

    instrument_id = order.instrument

    if order.instrument not in order_book:
        order_book[order.instrument] = OrderBook()

    order_book[order.instrument].add_order(order)

    order_book[instrument_id].sort_orders()

    # See if there are trades to be executed
    best_buy = order_book[instrument_id].get_best_buy()
    best_sell = order_book[instrument_id].get_best_sell()

    print(best_buy)
    print(best_sell)

    # 1. Buy and sell not market
    if not best_buy.is_market and not best_sell.is_market:
        # 1.a Buy price >= sell price
        if best_buy.price >= best_sell.price:
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True
            # Execute
        # 2.b Buy price < sell.price
            # Do not execute

    # 2. Buy is market and sell is not market
    elif best_buy.is_market:
        # Execute
        order_book[instrument_id].execute(best_buy, best_sell)
        trade_executed = True

    # 3. Sell is market and buy is not market
    elif best_sell.is_market:
        # Execute
        order_book[instrument_id].execute(best_buy, best_sell)
        trade_executed = True

    # 4. Buy and sell market
        # Sell non market and buy market

    else:
        if not (order_book[instrument_id].get_next_best_sell() is None):
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        elif not (order_book[instrument_id].get_next_best_buy() is None):
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        else:
            # All market orders
            pass

    print("###")
    print(order_book)
    print("###")

# Continue trading until no more trades possible
for instrument_id in order_book:
    order_book[instrument_id].sort_orders()

trade_executed = True

while trade_executed:
    trade_executed = False
    for instrument_id in order_book:
        order_book[instrument_id].sort_orders()
        best_buy = order_book[instrument_id].get_best_buy()
        best_sell = order_book[instrument_id].get_best_sell()

        # print(best_buy)
        # print(best_sell)

        # 1. Buy and sell not market
        if not best_buy.is_market and not best_sell.is_market:
            # 1.a Buy price >= sell price
            if best_buy.price >= best_sell.price:
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True
                # Execute
            # 2.b Buy price < sell.price
                # Do not execute

        # 2. Buy is market and sell is not market
        elif best_buy.is_market:
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        # 3. Sell is market and buy is not market
        elif best_sell.is_market:
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        # 4. Buy and sell market
            # Sell non market and buy market

        else:
            if not (order_book[instrument_id].get_next_best_sell() is None):
                # Execute
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True

            elif not (order_book[instrument_id].get_next_best_buy() is None):
                # Execute
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True

            else:
                # All market orders
                pass

        print("###")
        print(order_book)
        print("###")




# print(open_orders)

REJECTED - POSITION CHECK FAILED
{Order}: {'time': datetime.time(9, 5), 'order_id': 'C1', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.0, 'quantity': 100.0, 'is_market': False}
{Order}: {'time': datetime.time(9, 2), 'order_id': 'B1', 'client': {Client}: {'client_id': 'B', 'currencies': 'USD,SGD,JPY', 'position_check': False, 'rating': 2, 'positions': {'SIA': -500.0}}, 'instrument': 'SIA', 'side': 'Sell', 'price': 32.1, 'quantity': 4000.0, 'is_market': False}
###
{'SIA': ---{OrderBook}---
-buy
{Order}: {'time': datetime.time(9, 5), 'order_id': 'C1', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.0, 'quantity': 100.0, 'is_market': False}
{Order}: {'time': datetime.time(9, 29, 3), 'order_id': 'A2', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD

In [490]:
# Close auction (16:00 - 16:10)
close_orders = []
for order in orders:
    if (order.time > datetime.datetime.strptime("16:00:00", "%H:%M:%S").time()):
        if not validate_position(order):
            # Failed position check
            continue

        close_orders.append(order)


for order in close_orders:
    if order.instrument not in order_book:
        order_book[order.instrument] = OrderBook()

    order_book[order.instrument].add_order(order)

for instrument_id in order_book:
    order_book[instrument_id].sort_orders()

"""
Matching algorithm
"""

# 1. Buy and sell not market
    # 1.a Buy price >= sell price
        # Execute
    # 2.b Buy price < sell.price
        # Do not execute

# 2. Buy is market and sell is not market
    # Execute

# 3. Sell is market and buy is not market
    # Execute

# 4. Buy and sell market
    # Sell market and buy non market

trade_executed = True

while trade_executed:
    trade_executed = False
    for instrument_id in order_book:
        best_buy = order_book[instrument_id].get_best_buy()
        best_sell = order_book[instrument_id].get_best_sell()

        # print(best_buy)
        # print(best_sell)

        # 1. Buy and sell not market
        if not best_buy.is_market and not best_sell.is_market:
            # 1.a Buy price >= sell price
            if best_buy.price >= best_sell.price:
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True
                # Execute
            # 2.b Buy price < sell.price
                # Do not execute

        # 2. Buy is market and sell is not market
        elif best_buy.is_market:
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        # 3. Sell is market and buy is not market
        elif best_sell.is_market:
            # Execute
            order_book[instrument_id].execute(best_buy, best_sell)
            trade_executed = True

        # 4. Buy and sell market
            # Sell non market and buy market

        else:
            if not (order_book[instrument_id].get_next_best_sell() is None):
                # Execute
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True

            elif not (order_book[instrument_id].get_next_best_buy() is None):
                # Execute
                order_book[instrument_id].execute(best_buy, best_sell)
                trade_executed = True

            else:
                # All market orders
                pass

        print("###")
        print(order_book)
        print("###")


###
{'SIA': ---{OrderBook}---
-buy
{Order}: {'time': datetime.time(12, 30), 'order_id': 'E2', 'client': {Client}: {'client_id': 'E', 'currencies': 'SGD', 'position_check': False, 'rating': 5, 'positions': {'SIA': -900.0}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 31.9, 'quantity': 500.0, 'is_market': False}
{Order}: {'time': datetime.time(16, 5), 'order_id': 'A3', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {'SIA': 2300.0}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 31.9, 'quantity': 300.0, 'is_market': False}
{Order}: {'time': datetime.time(16, 9, 59), 'order_id': 'E3', 'client': {Client}: {'client_id': 'E', 'currencies': 'SGD', 'position_check': False, 'rating': 5, 'positions': {'SIA': -900.0}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 31.8, 'quantity': 2000.0, 'is_market': False}
-sell
{Order}: {'time': datetime.time(9, 40), 'order_id': 'B3', 'client': {Client}: {'client_id': 'B', 'currencies': 'USD,SGD

In [183]:
# Add orders to orderbook

# order_book = {}

# for order in orders:
#     if order.instrument not in order_book:
#         order_book[order.instrument] = OrderBook()

#     order_book[order.instrument].add_order(order)

# print(order_book)


{'SIA': ---{OrderBook}---
-buy
{Order}: {'time': datetime.time(9, 0, 1), 'order_id': 'A1', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 'Market', 'quantity': 1500.0}
{Order}: {'time': datetime.time(9, 5), 'order_id': 'C1', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.0, 'quantity': 100.0}
{Order}: {'time': datetime.time(9, 29, 3), 'order_id': 'A2', 'client': {Client}: {'client_id': 'A', 'currencies': 'USD,SGD', 'position_check': True, 'rating': 1, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 31.9, 'quantity': 800.0}
{Order}: {'time': datetime.time(10, 50), 'order_id': 'C3', 'client': {Client}: {'client_id': 'C', 'currencies': 'SGD', 'position_check': True, 'rating': 3, 'positions': {}}, 'instrument': 'SIA', 'side': 'Buy', 'price': 32.2